## Import Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import math
import pickle

from keras.models import Sequential
from keras.layers import GRU
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import Dense
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import L1L2

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error



## Define the functions to be used 

In [ ]:
from statistics import stdev

def Average(data):
    return sum(data) / len(data)

def normalize_series(data, mean, std):
    data = data - mean
    data = data / std
    return data

def inverse_normalize_series(data, mean, std):
    data = data * std
    data = data + mean
    return data

In [ ]:
def model_forecast(model, series, window_size, batch_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
   forecast = model.predict(ds)
   return forecast

# Dataset

## Read Dataset

In [ ]:
dataset = pd.read_excel('./dataset21Apr-21Jun2022.xlsx')
dataset = dataset[['date', 'pm2p5']]

In [ ]:
dataset.describe()

In [ ]:
plt.subplots(figsize = (15,6)) 
sns.lineplot(x="date", y="pm2p5",
             data=dataset, marker='.', color='red')
plt.title("Raw Data of Hourly Data PM2.5 from Apr 21st - Jun 21st 2022")

## Remove negative values and above 200

In [ ]:
for i in range(len(dataset)):
    if dataset['pm2p5'][i] < 0 or dataset['pm2p5'][i] > 200:
        dataset['pm2p5'][i] = np.nan

In [ ]:
dataset.describe()

In [ ]:
print(dataset.info())

In [ ]:
dataset['pm2p5'][1290:1320].plot()

## Fill empty data with interpolate method

In [ ]:
dataset['pm2p5'] = dataset['pm2p5'].interpolate(option='spline')

In [ ]:
print(dataset.info())

In [ ]:
dataset['pm2p5'][1290:1320].plot()

In [ ]:
from turtle import color


plt.subplots(figsize = (15,6))
sns.lineplot(x="date", y="pm2p5",
             data=dataset, marker='.')
plt.title("Hourly Data PM2.5 from Apr 21st - Jun 21st 2022")

In [ ]:
dataset.describe()

## Split data to validation and testing

split data with proportion 70% for training data, 20% for validation data and 10% for testing

In [ ]:
n_lookback = 24  # length of input sequences (lookback period)
n_forecast = 24  # length of output sequences (forecast period)

In [ ]:
SPLIT_TIME_TRAIN_REM = int(math.floor(0.7*((len(dataset))/24)) * 24) 
SPLIT_TIME_TRAIN_REM

In [ ]:
data_train = dataset[:SPLIT_TIME_TRAIN_REM]['pm2p5']
data_train = pd.DataFrame(data_train)
data_train.index = dataset[:SPLIT_TIME_TRAIN_REM]['date']

data_rem = dataset[SPLIT_TIME_TRAIN_REM:]['pm2p5']
data_rem = pd.DataFrame(data_rem)
data_rem.index = dataset[SPLIT_TIME_TRAIN_REM:]['date']

In [ ]:
SPLIT_TIME_VAL_TEST = int(math.floor(0.87*((len(data_rem))/24)) * 24) 
SPLIT_TIME_VAL_TEST

In [ ]:
data_val = data_rem[:SPLIT_TIME_VAL_TEST]
data_val = pd.DataFrame(data_val)

data_test = data_rem[SPLIT_TIME_VAL_TEST:]
data_test = pd.DataFrame(data_test)

In [ ]:
# % of training set
print(len(data_train)/len(dataset))
print(len(data_val)/len(dataset))
print(len(data_test)/len(dataset))

In [ ]:
data_test.head()

In [ ]:
from turtle import color


plt.subplots(figsize = (25,8))
sns.lineplot(x="date", y="pm2p5",
             data=data_train, marker='.', label='Training Data')
sns.lineplot(x="date", y="pm2p5",
             data=data_val, marker='.', label='Validation Data')
sns.lineplot(x="date", y="pm2p5",
             data=data_test, marker='.', label='Testing Data')
plt.title("Pembagian Data PM2.5 menjadi Data Pelatihan, Validasi, dan Pengujian", fontsize = 20)
plt.legend(loc='upper left', prop={'size': 16})

In [ ]:
pm2p5 = dataset["pm2p5"].values.reshape(-1, 1)

pm2p5_train = data_train.values.reshape(-1, 1)

pm2p5_val = data_val.values.reshape(-1, 1)

pm2p5_test = data_test.values.reshape(-1, 1)

print(pm2p5_train.shape);print(pm2p5_val.shape)

In [ ]:
x_train = []
y_train = []

for i in range(n_lookback, len(pm2p5_train) - n_forecast + 1):
    x_train.append(pm2p5_train[i - n_lookback: i])
    y_train.append(pm2p5_train[i: i + n_forecast])

x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)

In [ ]:
print(x_train[0]);print(y_train[0])

In [ ]:
x_val = []
y_val = []

for i in range(n_lookback, len(pm2p5_val) - n_forecast + 1):
    x_val.append(pm2p5_val[i - n_lookback: i])
    y_val.append(pm2p5_val[i: i + n_forecast])

x_val = np.array(x_val)
y_val = np.array(y_val)
print(x_val.shape)
print(y_val.shape)

In [ ]:
print(x_val[-1]);print(y_val[-1])

## Scaling / Normalize Data

In [ ]:
pm2p5_train_scaled = normalize_series(pm2p5_train, Average(pm2p5_train), stdev(pm2p5_train.flatten()))

pm2p5_val_scaled = normalize_series(pm2p5_val, Average(pm2p5_val), stdev(pm2p5_val.flatten()))

pm2p5_test_scaled = normalize_series(pm2p5_test, Average(pm2p5_test), stdev(pm2p5_test.flatten()))

## Set feature and label period

In [ ]:
x_train = []
y_train = []

for i in range(n_lookback, len(pm2p5_train_scaled) - n_forecast + 1):
    x_train.append(pm2p5_train_scaled[i - n_lookback: i])
    y_train.append(pm2p5_train_scaled[i: i + n_forecast])

x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_val = []
y_val = []

for i in range(n_lookback, len(pm2p5_val_scaled) - n_forecast + 1):
    x_val.append(pm2p5_val_scaled[i - n_lookback: i])
    y_val.append(pm2p5_val_scaled[i: i + n_forecast])

x_val = np.array(x_val)
y_val = np.array(y_val)
print(x_val.shape)
print(y_val.shape)

# Vanilla GRU

## generate model

In [ ]:
modelVanillaGRU = Sequential([
        GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True,
            input_shape=x_train[0].shape),                                          
        Flatten(),
        Dense(n_forecast)
    ])

modelVanillaGRU.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
modelVanillaGRU.compile(loss=tf.losses.Huber(),
              optimizer=optimizer,
              metrics=["mse"])

In [ ]:
estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=15,verbose=1,
                                       restore_best_weights=True)
mc=tf.keras.callbacks.ModelCheckpoint('./model v9/UnivariateForecast_VanillaGRU-Model.h5', monitor='val_mse', mode='min', verbose=1, save_best_only=True)

In [ ]:
historyVanillaGRU = modelVanillaGRU.fit(
    x_train, y_train, 
    epochs=1000, 
    batch_size=8, 
    callbacks=[estop, mc],
    validation_data=(x_val,y_val))

In [ ]:
with open('./model v9/historyVanillaGRU', 'wb') as file_pi:
    pickle.dump(historyVanillaGRU.history, file_pi)

with open('./model v9/historyVanillaGRU', "rb") as file_pi:
    load_historyVanillaGRU = pickle.load(file_pi)

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.title(f"Training vs Validation {metric} of Vanilla GRU")
    plt.legend([metric, f'val_{metric}'])
    plt.show()
    
plot_graphs(historyVanillaGRU, "loss")
plot_graphs(historyVanillaGRU, "mse")

## Prediction

### Prediction with training data

In [ ]:
forecast_train_data = model_forecast(modelVanillaGRU, pm2p5_train_scaled, n_lookback, 1)
print(forecast_train_data.shape)

forecast_train_data = forecast_train_data[:-1, 0]
print(forecast_train_data.shape)
print(pm2p5_train_scaled.shape[0]-n_lookback)

In [ ]:
forecast_train_data = inverse_normalize_series(forecast_train_data, Average(pm2p5_train), stdev(pm2p5_train.flatten()))
forecast_train_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_train[-forecast_train_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_train = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_train['Date'] = data_train[-forecast_train_data.shape[0]:].index
df_train['Forecast'] = forecast_train_data
df_train['Actual'] = actual
df_train

In [ ]:
plt.subplots(figsize=(20, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color = 'tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color = 'darkslateblue')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Latih (Train Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with validation data

In [ ]:
forecast_val_data = model_forecast(modelVanillaGRU, pm2p5_val_scaled, n_lookback, 1)
print(forecast_val_data.shape)

forecast_val_data = forecast_val_data[:-1, 0]
print(forecast_val_data.shape)
print(pm2p5_val_scaled.shape[0]-n_lookback)

In [ ]:
forecast_val_data = inverse_normalize_series(forecast_val_data, Average(pm2p5_val), stdev(pm2p5_val.flatten()))
forecast_val_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_val[-forecast_val_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
len(data_val[-forecast_val_data.shape[0]:])

In [ ]:
df_val = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_val['Date'] = data_val[-forecast_val_data.shape[0]:].index
df_val['Forecast'] = forecast_val_data
df_val['Actual'] = actual
df_val

In [ ]:
plt.subplots(figsize=(15, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Validasi (Validation Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with test data

In [ ]:
forecast_test_data = model_forecast(modelVanillaGRU, pm2p5_test_scaled, n_lookback, 1)
print(forecast_test_data.shape)

forecast_test_data = forecast_test_data[:-1, 0]
print(forecast_test_data.shape)
print(pm2p5_test_scaled.shape[0]-n_lookback)

In [ ]:
forecast_test_data = inverse_normalize_series(forecast_test_data, Average(pm2p5_test), stdev(pm2p5_test.flatten()))
forecast_test_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_test[-forecast_test_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_test = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_test['Date'] = data_test[-forecast_test_data.shape[0]:].index
df_test['Forecast'] = forecast_test_data
df_test['Actual'] = actual
df_test

In [ ]:


plt.subplots(figsize=(10, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data test aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data test", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Uji (Test Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

In [ ]:
fig, ax = plt.subplots(figsize = (25,8)) 
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color='tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color='darkslateblue')
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data uji aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data uji", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan\n", fontsize = 20)
plt.legend(loc='upper left')

## Summary Eval Metric

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

# 2-Stacked GRU

## generate model

In [ ]:
model2StackedGRU = Sequential([
        GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', 
            input_shape=x_train[0].shape, return_sequences=True),
        GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True),                                             
        Flatten(),
        Dense(n_forecast)
    ])

model2StackedGRU.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model2StackedGRU.compile(loss=tf.losses.Huber(),
              optimizer=optimizer,
              metrics=["mse"])

In [ ]:
estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=15,verbose=1,
                                       restore_best_weights=True)
mc=tf.keras.callbacks.ModelCheckpoint('./model v9/UnivariateForecast_2StackedGRU-Model.h5', monitor='val_mse', mode='min', verbose=1, save_best_only=True)

In [ ]:
history2StackedGRU = model2StackedGRU.fit(
    x_train, y_train, 
    epochs=1000, 
    batch_size=8, 
    callbacks=[estop, mc],
    validation_data=(x_val,y_val))

In [ ]:
with open('./model v9/history2StackedGRU', 'wb') as file_pi:
    pickle.dump(history2StackedGRU.history, file_pi)

with open('./model v9/history2StackedGRU', "rb") as file_pi:
    load_history2StackedGRU = pickle.load(file_pi)

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.title(f"Training vs Validation {metric} of 2-Stacked GRU")
    plt.legend([metric, f'val_{metric}'])
    plt.show()
    
plot_graphs(history2StackedGRU, "loss")
plot_graphs(history2StackedGRU, "mse")

## Prediction

### Prediction with training data

In [ ]:
def model_forecast(model, series, window_size, batch_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
   forecast = model.predict(ds)
   return forecast

In [ ]:
forecast_train_data = model_forecast(model2StackedGRU, pm2p5_train_scaled, n_lookback, 1)
print(forecast_train_data.shape)

forecast_train_data = forecast_train_data[:-1, 0]
print(forecast_train_data.shape)
print(pm2p5_train_scaled.shape[0]-n_lookback)

In [ ]:
forecast_train_data = inverse_normalize_series(forecast_train_data, Average(pm2p5_train), stdev(pm2p5_train.flatten()))
forecast_train_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_train[-forecast_train_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_train = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_train['Date'] = data_train[-forecast_train_data.shape[0]:].index
df_train['Forecast'] = forecast_train_data
df_train['Actual'] = actual
df_train

In [ ]:
plt.subplots(figsize=(20, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color = 'tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color = 'darkslateblue')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Latih (Train Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with validation data

In [ ]:
forecast_val_data = model_forecast(model2StackedGRU, pm2p5_val_scaled, n_lookback, 1)
print(forecast_val_data.shape)

forecast_val_data = forecast_val_data[:-1, 0]
print(forecast_val_data.shape)
print(pm2p5_val_scaled.shape[0]-n_lookback)

In [ ]:
forecast_val_data = inverse_normalize_series(forecast_val_data, Average(pm2p5_val), stdev(pm2p5_val.flatten()))
forecast_val_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_val[-forecast_val_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
len(data_val[-forecast_val_data.shape[0]:])

In [ ]:
df_val = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_val['Date'] = data_val[-forecast_val_data.shape[0]:].index
df_val['Forecast'] = forecast_val_data
df_val['Actual'] = actual
df_val

In [ ]:
plt.subplots(figsize=(15, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Validasi (Validation Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with test data

In [ ]:
forecast_test_data = model_forecast(model2StackedGRU, pm2p5_test_scaled, n_lookback, 1)
print(forecast_test_data.shape)

forecast_test_data = forecast_test_data[:-1, 0]
print(forecast_test_data.shape)
print(pm2p5_test_scaled.shape[0]-n_lookback)

In [ ]:
forecast_test_data = inverse_normalize_series(forecast_test_data, Average(pm2p5_test), stdev(pm2p5_test.flatten()))
forecast_test_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_test[-forecast_test_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_test = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_test['Date'] = data_test[-forecast_test_data.shape[0]:].index
df_test['Forecast'] = forecast_test_data
df_test['Actual'] = actual
df_test

In [ ]:
plt.subplots(figsize=(10, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data test aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data test", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Uji (Test Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

## Summary Eval Metric

In [ ]:
fig, ax = plt.subplots(figsize = (25,8)) 
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color='tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color='darkslateblue')
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data uji aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data uji", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan\n", fontsize = 20)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

# 3-Stacked GRU

## generate model

In [ ]:
model3StackedGRU = Sequential([
        GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True, 
            input_shape=x_train[0].shape),
        GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True),
        GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True),                                            
        Flatten(),
        Dense(n_forecast)
    ])

model3StackedGRU.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model3StackedGRU.compile(loss=tf.losses.Huber(),
              optimizer=optimizer,
              metrics=["mse"])

In [ ]:
estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=15,verbose=1,
                                       restore_best_weights=True)
mc=tf.keras.callbacks.ModelCheckpoint('./model v9/UnivariateForecast_3StackedGRU-Model.h5', monitor='val_mse', mode='min', verbose=1, save_best_only=True)

In [ ]:
history3StackedGRU = model3StackedGRU.fit(
    x_train, y_train, 
    epochs=1000, 
    batch_size=8, 
    callbacks=[estop, mc],
    validation_data=(x_val,y_val))

In [ ]:
with open('./model v9/history3StackedGRU', 'wb') as file_pi:
    pickle.dump(history3StackedGRU.history, file_pi)

with open('./model v9/history3StackedGRU', "rb") as file_pi:
    load_history3StackedGRU = pickle.load(file_pi)

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.title(f"Training vs Validation {metric} of 3-Stacked GRU")
    plt.legend([metric, f'val_{metric}'])
    plt.show()
    
plot_graphs(history3StackedGRU, "loss")
plot_graphs(history3StackedGRU, "mse")

## Prediction

### Prediction with training data

In [ ]:
def model_forecast(model, series, window_size, batch_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
   forecast = model.predict(ds)
   return forecast

In [ ]:
forecast_train_data = model_forecast(model3StackedGRU, pm2p5_train_scaled, n_lookback, 1)
print(forecast_train_data.shape)

forecast_train_data = forecast_train_data[:-1, 0]
print(forecast_train_data.shape)
print(pm2p5_train_scaled.shape[0]-n_lookback)

In [ ]:
forecast_train_data = inverse_normalize_series(forecast_train_data, Average(pm2p5_train), stdev(pm2p5_train.flatten()))
forecast_train_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_train[-forecast_train_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_train = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_train['Date'] = data_train[-forecast_train_data.shape[0]:].index
df_train['Forecast'] = forecast_train_data
df_train['Actual'] = actual
df_train

In [ ]:
plt.subplots(figsize=(20, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color = 'tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color = 'darkslateblue')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Latih (Train Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with validation data

In [ ]:
forecast_val_data = model_forecast(model3StackedGRU, pm2p5_val_scaled, n_lookback, 1)
print(forecast_val_data.shape)

forecast_val_data = forecast_val_data[:-1, 0]
print(forecast_val_data.shape)
print(pm2p5_val_scaled.shape[0]-n_lookback)

In [ ]:
forecast_val_data = inverse_normalize_series(forecast_val_data, Average(pm2p5_val), stdev(pm2p5_val.flatten()))
forecast_val_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_val[-forecast_val_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
len(data_val[-forecast_val_data.shape[0]:])

In [ ]:
df_val = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_val['Date'] = data_val[-forecast_val_data.shape[0]:].index
df_val['Forecast'] = forecast_val_data
df_val['Actual'] = actual
df_val

In [ ]:
plt.subplots(figsize=(15, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Validasi (Validation Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with test data

In [ ]:
pm2p5_test = data_test.values

pm2p5_test_scaled = normalize_series(pm2p5_test, Average(pm2p5_test), stdev(pm2p5_test.flatten()))

In [ ]:
forecast_test_data = model_forecast(model3StackedGRU, pm2p5_test_scaled, n_lookback, 1)
print(forecast_test_data.shape)

forecast_test_data = forecast_test_data[:-1, 0]
print(forecast_test_data.shape)
print(pm2p5_test_scaled.shape[0]-n_lookback)

In [ ]:
forecast_test_data = inverse_normalize_series(forecast_test_data, Average(pm2p5_test), stdev(pm2p5_test.flatten()))
forecast_test_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_test[-forecast_test_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_test = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_test['Date'] = data_test[-forecast_test_data.shape[0]:].index
df_test['Forecast'] = forecast_test_data
df_test['Actual'] = actual
df_test

In [ ]:
plt.subplots(figsize=(10, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data test aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data test", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Uji (Test Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

## Summary Eval Metric

In [ ]:
fig, ax = plt.subplots(figsize = (25,8)) 
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color='tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color='darkslateblue')
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data uji aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data uji", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan\n", fontsize = 20)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

# Biderectional GRU

## generate model

In [ ]:
# from numpy.random import seed
# seed(22)
# import tensorflow
# tensorflow.random.set_seed(22)

modelBiGRU = Sequential([
        Bidirectional(GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True),
                                          input_shape=x_train[0].shape),
        Flatten(),
        Dense(n_forecast)
    ])

modelBiGRU.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
modelBiGRU.compile(loss=tf.losses.Huber(),
              optimizer=optimizer,
              metrics=["mse"])

In [ ]:
estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=15,verbose=1,
                                       restore_best_weights=True)
mc=tf.keras.callbacks.ModelCheckpoint('./model v9/UnivariateForecast_BiGRU-Model.h5', monitor='val_mse', mode='min', verbose=1, save_best_only=True)

In [ ]:
historyBiGRU = modelBiGRU.fit(
    x_train, y_train, 
    epochs=1000, 
    batch_size=8, 
    # shuffle=True,
    callbacks=[estop, mc],
    validation_data=(x_val,y_val))

In [ ]:
with open('./model v9/historyBiGRU', 'wb') as file_pi:
    pickle.dump(historyBiGRU.history, file_pi)

with open('./model v9/historyBiGRU', "rb") as file_pi:
    load_historyBiGRU = pickle.load(file_pi)

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.title(f"Training vs Validation {metric} of BiGRU")
    plt.legend([metric, f'val_{metric}'])
    plt.show()
    
plot_graphs(historyBiGRU, "loss")
plot_graphs(historyBiGRU, "mse")

## Prediction

### Prediction with training data

In [ ]:
forecast_train_data = model_forecast(modelBiGRU, pm2p5_train_scaled, n_lookback, 1)
print(forecast_train_data.shape)

forecast_train_data = forecast_train_data[:-1, 0]
print(forecast_train_data.shape)
print(pm2p5_train_scaled.shape[0]-n_lookback)

In [ ]:
forecast_train_data = inverse_normalize_series(forecast_train_data, Average(pm2p5_train), stdev(pm2p5_train.flatten()))
forecast_train_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_train[-forecast_train_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_train = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_train['Date'] = data_train[-forecast_train_data.shape[0]:].index
df_train['Forecast'] = forecast_train_data
df_train['Actual'] = actual
df_train

In [ ]:
plt.subplots(figsize=(20, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color = 'tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color = 'darkslateblue')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Latih (Train Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with validation data

In [ ]:
forecast_val_data = model_forecast(modelBiGRU, pm2p5_val_scaled, n_lookback, 1)
print(forecast_val_data.shape)

forecast_val_data = forecast_val_data[:-1, 0]
print(forecast_val_data.shape)
print(pm2p5_val_scaled.shape[0]-n_lookback)

In [ ]:
forecast_val_data = inverse_normalize_series(forecast_val_data, Average(pm2p5_val), stdev(pm2p5_val.flatten()))
forecast_val_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_val[-forecast_val_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
len(data_val[-forecast_val_data.shape[0]:])

In [ ]:
df_val = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_val['Date'] = data_val[-forecast_val_data.shape[0]:].index
df_val['Forecast'] = forecast_val_data
df_val['Actual'] = actual
df_val

In [ ]:
plt.subplots(figsize=(15, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Validasi (Validation Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with test data

In [ ]:
forecast_test_data = model_forecast(modelBiGRU, pm2p5_test_scaled, n_lookback, 1)
print(forecast_test_data.shape)

forecast_test_data = forecast_test_data[:-1, 0]
print(forecast_test_data.shape)
print(pm2p5_test_scaled.shape[0]-n_lookback)

In [ ]:
forecast_test_data = inverse_normalize_series(forecast_test_data, Average(pm2p5_test), stdev(pm2p5_test.flatten()))
forecast_test_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_test[-forecast_test_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_test = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_test['Date'] = data_test[-forecast_test_data.shape[0]:].index
df_test['Forecast'] = forecast_test_data
df_test['Actual'] = actual
df_test

In [ ]:
plt.subplots(figsize=(10, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data test aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data test", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Uji (Test Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

## Summary Eval Metric

In [ ]:
fig, ax = plt.subplots(figsize = (25,8)) 
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color='tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color='darkslateblue')
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data uji aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data uji", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan\n", fontsize = 20)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

# 2 Stacked Biderectional GRU

## generate model

In [ ]:
model2StackedBiGRU = Sequential([
        Bidirectional(tf.keras.layers.GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True),
                                          input_shape=x_train[0].shape),
        Bidirectional(tf.keras.layers.GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True)),
        Flatten(),
        Dense(n_forecast)
    ])

model2StackedBiGRU.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model2StackedBiGRU.compile(loss=tf.losses.Huber(),
              optimizer=optimizer,
              metrics=["mse"])

In [ ]:
estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=15,verbose=1,
                                       restore_best_weights=True)
mc=tf.keras.callbacks.ModelCheckpoint('./model v9/UnivariateForecast_2StackedBiGRU-Model.h5', monitor='val_mse', mode='min', verbose=1, save_best_only=True)

In [ ]:
history2StackedBiGRU = model2StackedBiGRU.fit(
    x_train, y_train, 
    epochs=1000, 
    batch_size=8, 
    callbacks=[estop, mc],
    validation_data=(x_val,y_val))

In [ ]:
with open('./model v9/history2StackedBiGRU', 'wb') as file_pi:
    pickle.dump(history2StackedBiGRU.history, file_pi)

with open('./model v9/history2StackedBiGRU', "rb") as file_pi:
    load_history2StackedBiGRU = pickle.load(file_pi)

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.title(f"Training vs Validation {metric} of 2-Stacked BiGRU")
    plt.legend([metric, f'val_{metric}'])
    plt.show()
    
plot_graphs(history2StackedBiGRU, "loss")
plot_graphs(history2StackedBiGRU, "mse")

## Prediction

### Prediction with training data

In [ ]:
def model_forecast(model, series, window_size, batch_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
   forecast = model.predict(ds)
   return forecast

In [ ]:
forecast_train_data = model_forecast(model2StackedBiGRU, pm2p5_train_scaled, n_lookback, 1)
print(forecast_train_data.shape)

forecast_train_data = forecast_train_data[:-1, 0]
print(forecast_train_data.shape)
print(pm2p5_train_scaled.shape[0]-n_lookback)

In [ ]:
forecast_train_data = inverse_normalize_series(forecast_train_data, Average(pm2p5_train), stdev(pm2p5_train.flatten()))
forecast_train_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_train[-forecast_train_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_train = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_train['Date'] = data_train[-forecast_train_data.shape[0]:].index
df_train['Forecast'] = forecast_train_data
df_train['Actual'] = actual
df_train

In [ ]:
plt.subplots(figsize=(20, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color = 'tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color = 'darkslateblue')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Latih (Train Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with validation data

In [ ]:
forecast_val_data = model_forecast(model2StackedBiGRU, pm2p5_val_scaled, n_lookback, 1)
print(forecast_val_data.shape)

forecast_val_data = forecast_val_data[:-1, 0]
print(forecast_val_data.shape)
print(pm2p5_val_scaled.shape[0]-n_lookback)

In [ ]:
forecast_val_data = inverse_normalize_series(forecast_val_data, Average(pm2p5_val), stdev(pm2p5_val.flatten()))
forecast_val_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_val[-forecast_val_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
len(data_val[-forecast_val_data.shape[0]:])

In [ ]:
df_val = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_val['Date'] = data_val[-forecast_val_data.shape[0]:].index
df_val['Forecast'] = forecast_val_data
df_val['Actual'] = actual
df_val

In [ ]:
plt.subplots(figsize=(15, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Validasi (Validation Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with test data

In [ ]:
pm2p5_test = data_test.values

pm2p5_test_scaled = normalize_series(pm2p5_test, Average(pm2p5_test), stdev(pm2p5_test.flatten()))

In [ ]:
forecast_test_data = model_forecast(model2StackedBiGRU, pm2p5_test_scaled, n_lookback, 1)
print(forecast_test_data.shape)

forecast_test_data = forecast_test_data[:-1, 0]
print(forecast_test_data.shape)
print(pm2p5_test_scaled.shape[0]-n_lookback)

In [ ]:
forecast_test_data = inverse_normalize_series(forecast_test_data, Average(pm2p5_test), stdev(pm2p5_test.flatten()))
forecast_test_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_test[-forecast_test_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_test = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_test['Date'] = data_test[-forecast_test_data.shape[0]:].index
df_test['Forecast'] = forecast_test_data
df_test['Actual'] = actual
df_test

In [ ]:
plt.subplots(figsize=(10, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data test aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data test", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Uji (Test Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

In [ ]:
fig, ax = plt.subplots(figsize = (25,8)) 
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color='tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color='darkslateblue')
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data uji aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data uji", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan\n", fontsize = 20)
plt.legend(loc='upper left')

## Summary Eval Metric

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

# 3 Stacked Biderectional GRU

### Generate 3 Stacked BiGRU model

In [ ]:
model3StackedBiGRU = Sequential([
        Bidirectional(tf.keras.layers.GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True),
                                          input_shape=x_train[0].shape),
        Bidirectional(tf.keras.layers.GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True)),
        Bidirectional(tf.keras.layers.GRU(8, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', return_sequences=True)),
        Flatten(),
        Dense(n_forecast)
    ])

model3StackedBiGRU.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model3StackedBiGRU.compile(loss=tf.losses.Huber(),
              optimizer=optimizer,
              metrics=["mse"])

In [ ]:
estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=15,verbose=1,
                                       restore_best_weights=True)
mc=tf.keras.callbacks.ModelCheckpoint('./model v9/UnivariateForecast_3StackedBiGRU-Model.h5', monitor='val_mse', mode='min', verbose=1, save_best_only=True)

In [ ]:
history3StackedBiGRU = model3StackedBiGRU.fit(
    x_train, y_train, 
    epochs=1000, 
    batch_size=8, 
    callbacks=[estop, mc],
    validation_data=(x_val,y_val))

In [ ]:
with open('./model v9/history3StackedBiGRU', 'wb') as file_pi:
    pickle.dump(history3StackedBiGRU.history, file_pi)

with open('./model v9/history3StackedBiGRU', "rb") as file_pi:
    load_history3StackedBiGRU = pickle.load(file_pi)

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.title(f"Training vs Validation {metric} of 3-Stacked BiGRU")
    plt.legend([metric, f'val_{metric}'])
    plt.show()
    
plot_graphs(history3StackedBiGRU, "loss")
plot_graphs(history3StackedBiGRU, "mse")

## Prediction

### Prediction with training data

In [ ]:
def model_forecast(model, series, window_size, batch_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
   forecast = model.predict(ds)
   return forecast

In [ ]:
forecast_train_data = model_forecast(model3StackedBiGRU, pm2p5_train_scaled, n_lookback, 1)
print(forecast_train_data.shape)

forecast_train_data = forecast_train_data[:-1, 0]
print(forecast_train_data.shape)
print(pm2p5_train_scaled.shape[0]-n_lookback)

In [ ]:
forecast_train_data = inverse_normalize_series(forecast_train_data, Average(pm2p5_train), stdev(pm2p5_train.flatten()))
forecast_train_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_train[-forecast_train_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_train = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_train['Date'] = data_train[-forecast_train_data.shape[0]:].index
df_train['Forecast'] = forecast_train_data
df_train['Actual'] = actual
df_train

In [ ]:
plt.subplots(figsize=(20, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color = 'tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color = 'darkslateblue')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Latih (Train Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with validation data

In [ ]:
forecast_val_data = model_forecast(model3StackedBiGRU, pm2p5_val_scaled, n_lookback, 1)
print(forecast_val_data.shape)

forecast_val_data = forecast_val_data[:-1, 0]
print(forecast_val_data.shape)
print(pm2p5_val_scaled.shape[0]-n_lookback)

In [ ]:
forecast_val_data = inverse_normalize_series(forecast_val_data, Average(pm2p5_val), stdev(pm2p5_val.flatten()))
forecast_val_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_val[-forecast_val_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
len(data_val[-forecast_val_data.shape[0]:])

In [ ]:
df_val = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_val['Date'] = data_val[-forecast_val_data.shape[0]:].index
df_val['Forecast'] = forecast_val_data
df_val['Actual'] = actual
df_val

In [ ]:
plt.subplots(figsize=(15, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Validasi (Validation Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

### Prediction with test data

In [ ]:
forecast_test_data = model_forecast(model3StackedBiGRU, pm2p5_test_scaled, n_lookback, 1)
print(forecast_test_data.shape)

forecast_test_data = forecast_test_data[:-1, 0]
print(forecast_test_data.shape)
print(pm2p5_test_scaled.shape[0]-n_lookback)

In [ ]:
forecast_test_data = inverse_normalize_series(forecast_test_data, Average(pm2p5_test), stdev(pm2p5_test.flatten()))
forecast_test_data[:5]

In [ ]:
actual = np.squeeze(pm2p5_test[-forecast_test_data.shape[0]:])
print(actual.shape)
print(actual[:5])

In [ ]:
df_test = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_test['Date'] = data_test[-forecast_test_data.shape[0]:].index
df_test['Forecast'] = forecast_test_data
df_test['Actual'] = actual
df_test

In [ ]:
plt.subplots(figsize=(10, 5))
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data test aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data test", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan dari Data Uji (Test Data)\n", fontsize = 15)
plt.legend(loc='upper left')

In [ ]:
rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

In [ ]:
fig, ax = plt.subplots(figsize = (25,8)) 
ax = sns.lineplot(x="Date", y="Actual", data=df_train, 
                    label="data latih aktual", marker='.', color='tan')
ax = sns.lineplot(x="Date", y="Forecast", data=df_train, 
                    label="prakiraan data latih", marker='.', linestyle="--", color='darkslateblue')
ax = sns.lineplot(x="Date", y="Actual", data=df_val, 
                    label="data validasi aktual", marker='.', color='sandybrown')
ax = sns.lineplot(x="Date", y="Forecast", data=df_val, 
                    label="prakiraan data validasi", marker='.', linestyle="--", color='darkslategrey')
ax = sns.lineplot(x="Date", y="Actual", data=df_test,
                    label="data uji aktual", marker='.', color = 'skyblue')
ax = sns.lineplot(x="Date", y="Forecast", data=df_test, 
                    label="prakiraan data uji", marker='.', linestyle="--", color='darkmagenta')
plt.xlabel('Tanggal'); plt.ylabel('Konsentrasi PM 2.5')
plt.title("Grafik Data Aktual dan Hasil Prakiraan\n", fontsize = 20)
plt.legend(loc='upper left')

## Summary Eval Metric

In [ ]:
rmse = mean_squared_error(df_train['Forecast'], df_train['Actual'], squared=False)
mae = mean_absolute_error(df_train['Forecast'], df_train['Actual'])
mape = mean_absolute_percentage_error(df_train['Forecast'], df_train['Actual'])

print('Forecast Train accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_val['Forecast'], df_val['Actual'], squared=False)
mae = mean_absolute_error(df_val['Forecast'], df_val['Actual'])
mape = mean_absolute_percentage_error(df_val['Forecast'], df_val['Actual'])

print('Forecast Val accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))

rmse = mean_squared_error(df_test['Forecast'], df_test['Actual'], squared=False)
mae = mean_absolute_error(df_test['Forecast'], df_test['Actual'])
mape = mean_absolute_percentage_error(df_test['Forecast'], df_test['Actual'])

print('Forecast Test accuracy')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))
print('MAPE: ', round(mape,4))